<a href="https://colab.research.google.com/github/PrimePy/DataScience/blob/master/Deep%252520Learning/NLP/Auto_Text_Generator/auto_text_gen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Import Libs
import tensorflow as tf
import requests
import string

In [0]:
# Read the data
response = requests.get('https://ocw.mit.edu/ans7870/6/6.006/s08/lecturenotes/files/t8.shakespeare.txt')
data = response.text.split('\n')

In [0]:
# Remove header from the text file
data = data[253:]

In [0]:
# Clean the text data
data = ' '.join(data)

def clean_text(doc):
    tokens = data.split()
    punc = str.maketrans('', '', string.punctuation)
    tokens = [word.translate(punc) for word in tokens]
    tokens = [ word.lower() for word in tokens if word.isalpha()]
    return tokens

cleaned_tokens = clean_text(data)

In [0]:
# Convert tokens into sequence
seq_length = 50 + 1
sequences = []

for index in range(seq_length, len(cleaned_tokens)):
    line = ' '.join(cleaned_tokens[(index - seq_length) : index])
    sequences.append(line) 
    if index > 200000:
        break

In [0]:
# Build LSTM model and Preprocess input and output data
        
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer()
tokenizer.fit_on_texts(sequences)
tokenized_sequences = tokenizer.texts_to_sequences(sequences)
tokenized_sequences = np.array(tokenized_sequences[:])

In [8]:
# Initialize the x and y
x, y = tokenized_sequences[:, :-1], tokenized_sequences[:, -1]

# Initialize the vocabulary size
vocab_size = len(tokenizer.word_index) + 1

# Covert y into categorical
y = to_categorical(y, num_classes= vocab_size)

# Initialize the sequence length
seq_length = len(x[0])
print(len(x[0]))

50


In [9]:
# Create LSTM Model
model = Sequential()
model.add(Embedding(vocab_size, 50, input_length = seq_length))
model.add(LSTM(100, return_sequences = True))
model.add(LSTM(100))
model.add(Dense(100, activation = 'relu'))
model.add(Dense(vocab_size, activation = 'softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 50, 50)            1350      
_________________________________________________________________
lstm (LSTM)                  (None, 50, 100)           60400     
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense (Dense)                (None, 100)               10100     
_________________________________________________________________
dense_1 (Dense)              (None, 27)                2727      
Total params: 154,977
Trainable params: 154,977
Non-trainable params: 0
_________________________________________________________________


In [0]:
# Compile LSTM Model
model.compile( loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])


In [0]:
# Train LSTM Model
model.fit(x, y, batch_size = 256, epochs = 100)

Epoch 1/100
665/782 [========================>.....] - ETA: 48s - loss: 2.7059 - accuracy: 0.1836

In [0]:
vocab_size